# CBF Tutorial - Part 3: Higher-Order Barrier Functions

Authors: Bardh, Tom and Shakiba

## In this notebook, we consider higher-order barrier functions.

We start with a **unicycle** model, which has the following dynamics:

$$
\dot{x} = f(x) + g(x) \cdot u \\
$$

$$
\begin{bmatrix} \dot{xr_0} \\ \dot{xr_1} \\ \dot{xr_2} \end{bmatrix} = \begin{bmatrix} cos(xr_2) \\ sin(xr_2) \\ 0 \end{bmatrix} + \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix} u \tag{1}\\
$$

where the input to the system $u= \dot{xr_2} = \omega$ is the angular velocity.
We can imagine the robot's states are 2D positon with direction, $\begin{bmatrix} xr_0 & xr_1 & xr_2 \end{bmatrix}^T = \begin{bmatrix} x & y & \theta \end{bmatrix}^T$.

Given an initial position $x^{init}$, our goal is to control this system to a goal location $x^{goal}$.
$$
x^{init} = \begin{bmatrix} xr^{init}_0 \\  xr^{init}_1 \end{bmatrix} \qquad	x^{goal} = \begin{bmatrix} x^{goal}_0 \\  x^{goal}_1 \end{bmatrix} \tag{2}\\
$$
However, the system should avoid certain areas in the workspace (more on this later).

One way to solve this problem is to pose it as an optimization problem. Specifically, we would like to solve the following problem:
$$
\begin{align}
\min_{u} \quad & ||u_{ref} - u|| \\
\textrm{s.t.} \quad & \text{CBF constraints to be added later} \tag{3}
\end{align}
$$
So, we will try to minimize the distance between the decission variable $u$ and reference controller $u_{ref}$, while satisfying some constraints. For now, let us disregard the constraint and focus on designing a reference controller.

## Reference controller

We will define a very simple reference controller as follows
$$
u_{ref} = k \cdot (\arctan{(\frac{x^{goal}_0-xr_0}{x^{goal}_1-xr_1})} - xr_2) \tag{4}
$$
This is a two dimensional vector that takes the difference between the current system state (or location) and the goal location and multiply it with a gain factor $k$. This **simple** reference controller will take us from an initial location to the goal location. We can say it is a typical [proportional controller](https://en.wikipedia.org/wiki/Proportional_control). Next, we need to make sure that the system does not enter undesired states.

## Control Barrier Function (CBF) as Constraints

We assume that the undesired regions in the workspace can be defined with one or more [ellipses](https://en.wikipedia.org/wiki/Ellipse). In two dimensions, the region inside an ellypsis can be using the standard form with the following inequality
$$
\frac{z^2_0}{a^2} + \frac{z^2_1}{b^2}\leq 1 \tag{5}
$$
where $z_0,z_1$ define the origin and $a,b$ define the major, minor axes of the ellipse, respectively.

In our case, we would like to check whether our system is in the undesired region and therefore we modify the afforementined requirement
$$
B(x) = \frac{(x_0-z_0)^2}{a^2} + \frac{(x_1-z_1)^2}{b^2} - 1 \tag{6}
$$

In order to generate CBF, we need a more general theory of CBF. A CBF (AMES,2019) is defined as:

$$
L_fB(x) + L_gB(x)u + \alpha(B(x)) \ge 0
$$

Where $L_fB(x) = \frac{\partial B}{\partial x}^Tf(x)$ and $L_gB(x) = \frac{\partial B}{\partial x}^Tg(x)$ are the first order Lie derivatives of the system. This works well for first order systems, where the control input appears in the first derivative of $B$.

However, in our example, as we shall see, this function has a relative degree of 2 with respect to the system under consideration, since the first time that the control $u$ appears in the derivatives of B along the system dynamics is in its $2^{nd}$ derivative. In this case:

$$
L^2_fB(x) + L_gL_fB(x)u + \alpha_2( \dot{B}(x) + \alpha_1 B(x) ) \ge 0
$$

First,
$$
\dot{B}(x) = \frac{\partial{B}}{\partial{x}} \cdot \frac{\partial{x}}{\partial{t}} = \begin{bmatrix} (-2z_0 + 2xr_0)/a^2] \\ (-2z_1 + 2xr_1)/b^2] \\ 0 \end{bmatrix}^T \cdot \dot{x}
$$

Then, we caluclate
$$
L^2_fB(x) = ((-2z_0 + 2xr_0)/a^2 + 2cos(xr_2)/a^2)*cos(xr_2) + ((-2z_1 + 2xr_1)/b^2 + 2sin(xr_2)/b^2) \cdot sin(xr_2)
$$
<!-- L^2_fB(x) = (\alpha_1*(-2z_0 + 2xr_0)/a^2 + 2cos(xr_2)/a^2)*cos(xr_2) + (\alpha_1*(-2z_1 + 2xr_1)/b^2 + 2sin(xr_2)/b^2)*sin(xr_2) -->
After, we calculate
$$
L_gL_fB(x) = ((-2z_1 + 2xr_1)*cos(xr_2)/b^2 - (-2z_0 + 2xr_0)*sin(xr_2)/a^2)*\dot{xr_2}
$$

Here, we notice that our control input $\dot{xr_2}$ appears in the expression and we can therefore continue in the next step of synthesizing a controller.

## Posing the control problem as a quadratic program

We can now combine the cost function with the constraints to form an optimization problem.
$$
\begin{align}
\min_{u} \quad & ||u_{ref} - u||\\ \textrm{s.t.} \quad & L_gL_fB(x)u  \ge -L^2_fB(x) - \alpha_2( \dot{B}(x) + \alpha_1 B(x) ) 
\end{align}
$$

This problem can be posed as a [quadratic problem](https://en.wikipedia.org/wiki/Quadratic_programming) of the form
$$
\begin{align}
\min_{\mathbf{x}} \quad & \frac{1}{2}\mathbf{x}^T P\mathbf{x} + q^T\mathbf{x} \\
\textrm{s.t.} \quad & G\mathbf{x} \le h \tag{13}
\end{align}
$$
In our case, $\mathbf{x} = \dot{x}$ is the decision variable,
Where

$$
P = 1 \tag{14}
$$
$$
q =  u_{ref0} \tag{15}
$$

$$
G = L_gL_fB(x) \tag{16}
$$
$$
h = -L^2_fB(x) - \alpha_2( \dot{B}(x) + \alpha_1 B(x) ) \tag{17}
$$


Here, one of advantage of this methodology is that instance of controller is not needed, as we can see (15) referes only result of controller reference $u_{ref}$. In another word, the CBF can work as an observer in whole controll system.

## Code implementation

First, we import the necessary packages for our example.


In [ ]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as cvxopt
from matplotlib.patches import Ellipse
from sympy import symbols, Matrix, sin, cos
import matplotlib.animation as animation
from cbflib import cbf, cbf_utils, sys_and_ctrl

In [ ]:
# Robot Goal
x_goal = np.array([5, 5])

# Undesired areas in ellipse format (x,y,rad_x,rad_y) - Use example(0) through example(3)
bad_sets = cbf_utils.example(3)

# Parameters for reference controller
ctrl_param = [10]

# Symbols and equations for the CBF
xr0, xr1, xr2, cx, cy, rad_x, rad_y, xr2_dot, u = symbols(
    'xr0 xr1 xr2 cx cy rad_x rad_y xr2_dot u')
symbs = (cx, cy, rad_x, rad_y, xr0, xr1, xr2)

# Barrier function - distance of robot to obstacle
B = ((xr0 - cx)/rad_x)**2 + ((xr1 - cy)/rad_y)**2 - 1

# dx = f(x) + g(x)u
f = Matrix([cos(xr2), sin(xr2), 0])
g = Matrix([0, 0, 1])
states_dot = Matrix([cos(xr2), sin(xr2), xr2_dot])

# Initialize CBF
myCBF = cbf.CBF(B=B, f=f, g=g, symbs = symbs, states=(xr0, xr1, xr2),
                bad_sets=bad_sets, states_dot=states_dot, degree=2)

#? Simulation settings
T_max = 10
n_samples = 500
T = np.linspace(0, T_max, n_samples)
dt = T[1]-T[0]
params = {'goal_x': x_goal, 'bad_sets': bad_sets,
          'ctrl_param': ctrl_param, 'myCBF': myCBF}

cvxopt.solvers.options['show_progress'] = False

x_0 = np.array([0.5, 0.5, 0])

In [ ]:
# Loop through initial conditions
print('\nComputing trajectories for the initial condition:')
print('x_0\t x_1')
print(x_0[0], '\t', x_0[1])

# If initial condition is inside the bad set, error out.
curr_bs = []
for idxj, j in enumerate(bad_sets):
    curr_bs = bad_sets[idxj]
    assert cbf_utils.is_inside_ellipse(
        [x_0[0], x_0[1]], bad_sets[idxj]) == 0, "Initial condition is inside ellipse"


# Compute output on the unicycle system for given initial conditions and timesteps T
x = np.zeros((np.size(x_0), len(T)))
x[:, 0] = x_0
for i in range(len(T)-1):
    x[:, i+1] = x[:, i] + dt * \
        np.array(sys_and_ctrl.unicycle_f(T[i], x[:, i], [], params))

In [ ]:
# Animate
fig, ax = plt.subplots(figsize=(8,8))
ax = cbf_utils.plot_cbf_elements(ax, bad_sets, x_goal)

plt.xlim(-2, 7)
plt.ylim(-2, 7)

line1, = ax.plot([], [], lw=2)
goal_square = plt.Rectangle(
    x_goal-np.array([.5, .5]), .2, .2, color='r', alpha=0.5)

def init():
    line1.set_data([], [])
    return line1


def animate(i):
    line1.set_data((x[0][0:i], x[1][0:i]))
    return line1, goal_square

plt.close()

ani = animation.FuncAnimation(
    fig, animate, init_func=init, interval=10, frames=n_samples, repeat=True, blit=True)

from IPython.display import HTML
from matplotlib import rcParams
rcParams['animation.html'] = 'html5'
HTML(ani.to_html5_video())
ani

To run the code directly in python see [03-Tutorial-HOCFB-Static-Obstacles.py](./03-Tutorial-HOCFB-Static-Obstacles.py)